# New York Stock Exchange

Long have analysts, traders, and investors tried to predict future stock prices. Even if you come up with a plan or model that works for a while, it is generally great until for some reason it doesn't work. Whether there was a break down in underlying correlations or new news of scandal caused stock prices to plummet. If you could crack the whole code you'd be a billionaire in about the same amount of time as it takes to click on your favorite Netflix program. 

If any type of model is going to be able to finally do it, I would be willing to bet that model would fall into the deep learning category. So that is exactly what we are going to try to do.  


## The Data
The data set we will be working with is from the New York Stock Exchange (NYSE) and represent the historical prices and other fundamental data points of the S&P 500 from 2010 to the end of 2016.

Dataset consists of following files:

prices.csv: raw, as-is daily prices. Most of data spans from 2010 to the end of 2016, for companies new on stock market date range is shorter. There have been approx. 140 stock splits in that time, this set doesn't account for that.

prices-split-adjusted.csv: same as prices, but there have been added adjustments for splits. securities.csv: general description of each company with division on sectors

fundamentals.csv: metrics extracted from annual SEC 10K filings (2012-2016), should be enough to derive most of popular fundamental indicators.

The majority of our focus will be on the prices-split-adjusted.csv file, as this contains the adjusted prices for the stocks we will be trying to predict.

## The Process

This project will statisfy the third project requirement for the Data Science Bootcamp at the Flatiron School

In General, we will follow the Data Science process, Cleaning, Exploring, and Analyzing iteratively until we arrive at a useable model.  

As always we can begin by importing all the necessary libraries for our notebook.

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.models import Sequential
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='whitegrid')
sns.set_palette('Set1')
seed = 7
np.random.seed(seed)

Using TensorFlow backend.
